In [47]:
import numpy as np
!pip install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [48]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
    
# pointers
UP = (-1, 0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

In [49]:
Sigma = ['A', 'C', 'T', 'G']
seqLength = 100
seqA = np.random.choice(Sigma,seqLength)
seqB = np.random.choice(Sigma,seqLength)
# seqA,seqB
len(seqA),len(seqB)

(100, 100)

## Needleman

In [50]:
def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])
    

def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = np.zeros((len(v)+1,len(w)+1))
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None
    for i in range(len(v)+1):
      for j in range(len(w)+1):
        if i == 0 and j == 0:
          continue
        if i == 0 and j > 0:
          M[i][j] = M[i][j-1] + delta['-'][w[j-1]]
          pointers[i][j] = LEFT
          continue
        if j == 0 and i > 0:
          M[i][j] = M[i-1][j] + delta[v[i-1]]['-']
          pointers[i][j] = UP
          continue
        up = M[i-1][j] + delta[v[i-1]]['-']
        left = M[i][j-1] + delta['-'][w[j-1]]
        topleft = M[i-1][j-1] + delta[v[i-1]][w[j-1]]
        if left >= up and left >= topleft:
          M[i][j] = left
          pointers[i][j] = LEFT
        elif up >= left and up >= topleft:
          M[i][j] = up
          pointers[i][j] = UP
        else:
          M[i][j] = topleft
          pointers[i][j] = TOPLEFT
    score = M[i][j]
    alignment = traceback_global(v,w, pointers)
    return score, alignment

In [51]:
%prun score, alignment = global_align(seqA, seqB, delta)
print(score)
print(alignment)

 1.0
AGGTACTCCA-GCACCCGGTAAGAGTG-AGGTCTGGCGCACAT--AAGG-CCTA-CAATGAATGAC-TCTG-TCATGCG-AA-C-ATCG-GC--A-GT--TCCAA--TAT-CAATTTCGC-
-G-T-C-C-ATGCGT--GGTC-G-GTGTAA-T-T--C-CTTTTTTAACTTCCTAGCA-T-AATTACGTCCACTC-T-CGGAAACTAT-GAGCTTATGTGGTT-AGCGTGTACAATAA-GTA


## Hirschberg

In [52]:
def last_col(v, w, delta):
    """
    Returns the scores of the last column of alignment of the strings v and w
    
    :param: v
    :param: w
    :param: delta
    """
    pre_col = np.zeros(len(v)+1,int)
    cur_col = np.zeros(len(v)+1,int)

    for i in range(1,len(v)+1):
      pre_col[i] = pre_col[i-1] + delta[v[i-1]]['-']

    for j in range(1,len(w)+1):
      cur_col[0] = pre_col[0] + delta['-'][w[j-1]]
      for i in range(1,len(v)+1):
        up = cur_col[i-1] + delta[v[i-1]]['-']
        left = pre_col[i] + delta['-'][w[j-1]]
        topleft = pre_col[i-1] + delta[v[i-1]][w[j-1]]
        cur_col[i] = max(up,left,topleft)
      # print(pre_col)
      # print(cur_col)
      pre_col = cur_col
      cur_col = np.zeros(len(v)+1)
    return pre_col

In [53]:
col = last_col("ATGTC", "AT", delta)
print(col)

[-2.  0.  2.  1.  0. -1.]


In [54]:
def Hirschberg(v,w,delta):
  def HB(i,j,k,l):
    if l-j <= 1: return
    mid = j + (l-j) // 2
    prefix = last_col(v[i:k],w[j:mid],delta)
    suffix = last_col(v[i:k][::-1],w[mid:l][::-1],delta)
    i_star = i + np.argmax(prefix + suffix[::-1])
    align.append((i_star,mid))
    HB(i,j,i_star,mid)
    HB(i_star,mid,k,l)
    
  align = [(len(v),len(w))]
  HB(0,0,len(v),len(w))
  align.sort()
  
  score = 0.0
  new_v = []
  new_w = []
  prev = (0,0)
  for (i,j) in align:
    if tuple(map(lambda x,y:x-y, prev,(i,j))) == LEFT:
      score += delta['-'][w[j-1]]
      new_v.append('-')
      new_w.append(w[j-1])
    elif tuple(map(lambda x,y:x-y, prev,(i,j))) == UP:
      score += delta[v[i-1]]['-']
      new_v.append(v[i-1])
      new_w.append('-')
    elif tuple(map(lambda x,y:x-y, prev,(i,j))) == TOPLEFT:
      score += delta[v[i-1]][w[j-1]]
      new_v.append(v[i-1])
      new_w.append(w[j-1])
    else:
      while tuple(map(lambda x,y:x-y, prev,(i,j))) != TOPLEFT:
        score += delta[v[prev[0]]]['-']
        new_v.append(v[prev[0]])
        new_w.append('-')
        prev = (prev[0]+1,prev[1])
        # print(score)
      score += delta[v[i-1]][w[j-1]]
      new_v.append(v[i-1])
      new_w.append(w[j-1])      
    prev = (i,j)
    # print(score)
  return score, ''.join(new_v)+'\n'+''.join(new_w)

In [55]:
%prun score,alignment = Hirschberg(seqA, seqB, delta)
print(score)
print(alignment)

 1.0
AGGTACTCCA-GCACCCGGTAAGAGTG-AGGTCTGGCGCACATAA-GGCCTA-CAATGAATGAC-T-CTGTCAT-GCG-AAC-ATCG-GC--A----GTT--CCAATATCAATTTCG-C
-G-T-C-C-ATGCGT--GGTC-G-GTGTAATTC---CTTTTTTAACTTCCTAGCA-T-AATTACGTCCACTC-TCG-GAAACTAT-GAGCTTATGTGGTTAGCGTGTA-CAATAA-GTA


## Memory Usage Trace with %mprun

In [56]:
Sigma = ['A', 'C', 'T', 'G']
seqLength = 100
seqA = np.random.choice(Sigma,seqLength)
seqB = np.random.choice(Sigma,seqLength)
# seqA,seqB
len(seqA),len(seqB)

(100, 100)

In [57]:
from Needleman import global_align
%mprun -f global_align global_align(seqA, seqB, delta)

The length of the input sequences are: n =  100 , m =  100



In [58]:
from Hirschberg import Hirschberg
%mprun -f Hirschberg Hirschberg(seqA, seqB, delta)

The length of the input sequences are: n =  100 , m =  100

